In [ ]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-tl7h67fb
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-tl7h67fb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 16.7MB/s 
     |████████████████████████████████| 3.3MB 30.0MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2097542 sha256=3371c412f48c6d6d485022525f79c58c5ec519a8257855b3749387a52959aaf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-bvw4bsj0/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers
     |████████████████████████████████| 225kB 14.2MB/s 
     |████████████████████████████████| 112kB 21.9MB/s 
     |████████████████████████████████| 245kB 26.8MB/s 


In [ ]:
import torch
import numpy
import matplotlib.pyplot as plt
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import argparse
import pdb

from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import RobertaTokenizer, RobertaForMaskedLM
from transformers import Trainer, TrainingArguments


# print(torch.cuda.device_count())
# print(torch.cuda.get_device_name(0))

In [ ]:
args_dict = {
    "train_dataset": "/content/drive/MyDrive/OMSCS_DL/CITATION/data/train.jsonl",
    "val_dataset": "/content/drive/MyDrive/OMSCS_DL/CITATION/data/dev.jsonl",
    "test_dataset": "/content/drive/MyDrive/OMSCS_DL/CITATION/data/test.jsonl",
    "model": "roberta-base",
    "output_dir": "/content/drive/MyDrive/OMSCS_DL/CITATION/output/mlm_tune/mlm_pretrain/",
    "adapter_name": "adapter_CITATION_"
}

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

In [ ]:
train_data = datasets.load_dataset(
                "json", data_files=args_dict["train_dataset"])["train"]
#not a typo: load_dataset loads this as training data
val_data = datasets.load_dataset(
                "json", data_files=args_dict["val_dataset"])["train"] 

Using custom data configuration default-e6a7c44083f2214b
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-e6a7c44083f2214b/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02)
Using custom data configuration default-bc5e6cf9187051d7


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-bc5e6cf9187051d7/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [ ]:
def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding=True, truncation=True, return_special_tokens_mask=True)
    return tokenized_batch

train_data = train_data.map(tokenization, batched=True, batch_size=len(train_data), remove_columns=["text", "label"])
train_data.set_format('torch', columns=['input_ids'])


val_data = val_data.map(tokenization, batched=True, batch_size=len(train_data), remove_columns=["text", "label"])
val_data .set_format('torch', columns=['input_ids'])

In [ ]:
#static masking. Possible TODO: use dynamic masking
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)



In [ ]:
adapter_name = "mlm_CITAION_pretrain"
model.add_adapter(adapter_name)
model.train_adapter(adapter_name)

ValueError: ignored

In [ ]:
training_args = TrainingArguments(
    num_train_epochs=150,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=1e-4,
    #lr_scheduler_type="constant",        
    adam_epsilon=1e-6,
    adam_beta1=0.9,
    adam_beta2=0.98,
    weight_decay=0.01,
    warmup_ratio=0.06,
    fp16=True,
    eval_accumulation_steps=1,
    save_steps=5000,
    save_total_limit=2,
    overwrite_output_dir=True,
    remove_unused_columns=False,
    seed=2,
    output_dir="/content/drive/MyDrive/OMSCS_DL/CITATION/output/mlm_tune/mlm_pretrain",
    evaluation_strategy = "epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=val_data
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,No log,6.953455,1.275600,89.369000
2,No log,6.782236,1.282200,88.909000
3,No log,6.793384,1.275900,89.347000
4,No log,6.472751,1.279000,89.133000
5,No log,6.311337,1.281500,88.959000
6,No log,6.323278,1.286300,88.629000
7,No log,6.304282,1.268800,89.846000
8,No log,6.267212,1.264800,90.130000
9,No log,6.164958,1.249300,91.254000
10,7.602500,6.069895,1.256200,90.749000


TrainOutput(global_step=7950, training_loss=4.076732599990173, metrics={'train_runtime': 9052.9413, 'train_samples_per_second': 0.878, 'total_flos': 5.3423806002336e+16, 'epoch': 150.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 815104, 'train_mem_gpu_alloc_delta': 12466176, 'train_mem_cpu_peaked_delta': 0, 'train_mem_gpu_peaked_delta': 12622582272})

In [ ]:
 model.save_adapter(args_dict["output_dir"] + adapter_name, adapter_name)